In [1]:
import random
import numpy as np

**Perceptron Class**

Here we create a class for the Perceptron that we may use to instatiate any number of Perceptron instances.

In [2]:
class Perceptron:
    
    def __init__(self, num_inputs, act='sigmoid'):
        self.weights = []
        self.num_inputs = num_inputs
        self.act = act # define activation function with sigmoid being the default
        for x in range(0, num_inputs):
            self.weights.append(random.random() * 2 - 1)
        print(self.weights)
            
    def get_weights(self):
        return self.weights
        
    def feed_forward(self, inputs):
        self.inputs = inputs
        sum = 0
        
        # multiply inputs by weights and sum them
        for i in range(0, self.num_inputs):
            sum += self.weights[i] * inputs[i]
            
        # 'activate' the sum and get the derivative
        self.output, self.output_prime = self.activate(sum)
        return self.output
    
    def activate(self, x):
        if (self.act == 'sigmoid'):
            activation = self.sigmoid(x)
            activation_prime = activation * (1 - activation)
        else:
            activation = self.step(x)
            activation_prime = 1 # use 1 since step activation is not differentiable
        return activation, activation_prime
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
    
    def step(self, x):
        if x > 0:
            return 1
        return 0
    
    def backward_pass(self, error):
        learning_rate = 0.01 # hyperparameter
        back_error = [] # each element in list represent amount of error to send backward along that connection
        for i in range(0, self.num_inputs):
            back_error.append(error * self.output_prime * self.weights[i])
            self.weights[i] -= error * self.output_prime * self.inputs[i] * learning_rate
        return back_error

**Single Perceptron to classify a point as being above or below a line**

Begin with a line going through the origin.

In [44]:
def line(x):
    return 0.5 * x

In [56]:
p = Perceptron(2, act='step')
weights = p.get_weights()
print(sum(weights))

[-0.08058198345153089, 0.9651201836932843]
0.8845382002417534


In [79]:
# pick 1 random point and determine if it's above or below the line
x_coord = random.random() * 1000
y_coord = random.random() * 1000
line_y = line(x_coord)

print(x_coord, y_coord)
print(x_coord, line_y)

if y_coord > line_y:
    answer = 1
else:
    answer = 0
    
print(answer)

901.9672274233975 255.103627952859
901.9672274233975 450.98361371169875
0


In [80]:
# pass the random point into the node, feed it back and see if the weights change
guess = p.feed_forward([x_coord, y_coord])
p.backward_pass(guess - answer)
p.get_weights()

[-8.685332254545209, -2.38793593594849]


[-8.685332254545209, -2.38793593594849]

In [57]:
# train the network
for _ in range(0, 1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    if y_coord > line_y:
        answer = 1
    else:
        answer = 0
    
    guess = p.feed_forward([x_coord, y_coord])
    p.backward_pass(guess - answer)
    
p.get_weights()

[-18.29384246820456, 36.81655593994777]

In [58]:
# check the accuracy
correct = 0

for _ in range(0,1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    is_above = y_coord > line_y
    guess_above = p.feed_forward([x_coord, y_coord, 1])
    
    if (is_above == True and guess_above >= 0.5):
        correct += 1
    if (is_above == False and guess_above < 0.5):
        correct += 1

print(correct)

999


Add bias to the node in order to handle lines that do not go through the origin.

In [71]:
def line(x):
    return 0.5 * x + 500

In [78]:
p = Perceptron(3, act='step')
weights = p.get_weights()

[0.2214871748797087, 0.17398763304982867, -0.5255616218064509]


In [79]:
# train the Perceptron using a bias term
for _ in range(0, 100000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    if y_coord > line_y:
        answer = 1
    else:
        answer = 0
    
    guess = p.feed_forward([x_coord, y_coord, 1])
    p.backward_pass(guess - answer)
    
p.get_weights()

[-15.181007899220747, 13.873452267396452, -65.51556162180306]

In [80]:
# determine the accuracy
correct = 0

for _ in range(0,1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    above = y_coord > line_y
    guess_above = p.feed_forward([x_coord, y_coord, 1])
    
    if (above == True and guess_above >= 0.5):
        correct += 1
    if (above == False and guess_above < 0.5):
        correct += 1

print(correct)

800


**Perceptrons for Logic**

Implement logicial AND and OR

In [58]:
a = Perceptron(3) # logical AND
b = Perceptron(3) # logical OR

[0.1973171969546439, 0.9407410789339032, -0.8785536451209432]
[0.8440494165572345, -0.7232351403105719, 0.1247101699302735]


In [65]:
# Train the 'a' Perceptron for logical AND
for _ in range(0,100000):
    first = random.choice([0, 1])
    second = random.choice([0, 1])
    a_out = a.feed_forward([first, second, 1])
    
    if (first and second):
        answer = 1
    else:
        answer = 0
    a.backward_pass(a_out - answer)
    
print(a.get_weights())

# Train the 'b' Perceptron for logical OR
for _ in range(0,100000):
    first = random.choice([0, 1])
    second = random.choice([0, 1])
    b_out = b.feed_forward([first, second, 1])
    
    if (first or second):
        answer = 1
    else:
        answer = 0
    b.backward_pass(b_out - answer)
    
print(b.get_weights())

[5.153930989203262, 5.152757919753352, -7.824237161307208]
[5.854616254553259, 5.851694138766977, -2.675989289602662]


In [66]:
print("Logical AND")
print(a.feed_forward([1, 1, 1]))
print(a.feed_forward([1, 0, 1]))
print(a.feed_forward([0, 1, 1]))
print(a.feed_forward([0, 0, 1]))
print("\nLogical OR")
print(b.feed_forward([1, 1, 1]))
print(b.feed_forward([1, 0, 1]))
print(b.feed_forward([0, 1, 1]))
print(b.feed_forward([0, 0, 1]))

Logical AND
0.922902429355317
0.06474842555742528
0.06467742534473865
0.00039976367222106004

Logical OR
0.9998802902889667
0.9600220025673405
0.9599097015938103
0.06440512841264989


**XOR with 2 Perceptrons**

In [21]:
a = Perceptron(3, act='step')
b = Perceptron(4, act='step')

[0.44233208077511255, 0.46563241559334, -0.9488606598636238]
[0.09891392173295421, -0.943930983402608, 0.3970823323953001, 0.48435790289690983]


In [22]:
def network(first, second):
    a_out = a.feed_forward([first, second, 1])
    b_out = b.feed_forward([first, a_out, second, 1])
    return b_out

In [24]:
for _ in range(0,1000):
    first = random.choice([0, 1])
    second = random.choice([0, 1])
    
    a_out = a.feed_forward([first, second, 1])
    b_out = b.feed_forward([first, a_out, second, 1])
    
    if (first != second):
        answer = 1
    else:
        answer = 0
        
    back_error = b.backward_pass(b_out - answer)
    a.backward_pass(back_error[1])
    
print(a.get_weights())
print(b.get_weights())

[0.4517713906091386, 0.4750717254273661, -0.486334477996346]
[0.08891392173295422, -0.943930983402608, 0.3870823323953001, -0.005642097103090453]


In [26]:
print(network(1, 1))
print(network(1, 0))
print(network(0, 1))
print(network(0, 0))

0
1
1
0


**XOR Network with 3 Perceptrons**

In [127]:
a = Perceptron(3, act='step')
b = Perceptron(3, act='step')
c = Perceptron(3, act='step')

[-0.4174712053572198, -0.4898950878980841, -0.1779856508107156]
[0.8072557493479546, 0.5407030478363373, 0.4910981953577074]
[-0.7345969723665828, 0.03912538710677449, -0.3920147667766811]


In [128]:
def network(first, second):
    a_out = a.feed_forward([first, second, 1])
    b_out = b.feed_forward([first, second, 1])
    c_out = c.feed_forward([a_out, b_out, 1])
    return c_out

In [129]:
# train the network
for _ in range(0,1000000):
    first = random.choice([0, 1])
    second = random.choice([0, 1])
    c_out = network(first, second)
#     a_out = a.feed_forward([first, second, 1])
#     b_out = b.feed_forward([first, second, 1])
#     c_out = c.feed_forward([a_out, b_out, 1])
    
    if (first != second):
        answer = 1
    else:
        answer = 0
        
    back_error = c.backward_pass(c_out - answer)
    a.backward_pass(back_error[0])
    b.backward_pass(back_error[1])
    
print(a.get_weights())
print(b.get_weights())
print(c.get_weights())

[0.00021443308871410462, 0.32926158170515585, -0.3294429544550295]
[0.3542379474046992, 0.02896512201889478, -0.003002965607727576]
[-0.1345969723665823, 0.2591253871067746, -0.16201476677668092]


In [130]:
print(network(1, 1))
print(network(1, 0))
print(network(0, 1))
print(network(0, 0))

0
1
1
0


In [131]:
print('%f' % a.feed_forward([1, 1, 1]))
print('%f' % a.feed_forward([1, 0, 1]))
print('%f' % a.feed_forward([0, 1, 1]))
print('%f' % a.feed_forward([0, 0, 1]))
print("\n")
print('%f' % b.feed_forward([1, 1, 1]))
print('%f' % b.feed_forward([1, 0, 1]))
print('%f' % b.feed_forward([0, 1, 1]))
print('%f' % b.feed_forward([0, 0, 1]))
print("\n")
print('%f' % c.feed_forward([1, 1, 1]))
print('%f' % c.feed_forward([1, 0, 1]))
print('%f' % c.feed_forward([0, 1, 1]))
print('%f' % c.feed_forward([0, 0, 1]))

1.000000
0.000000
0.000000
0.000000


1.000000
1.000000
1.000000
0.000000


0.000000
0.000000
1.000000
0.000000


**Parabola**

Utilizing non-lenear activation

In [45]:
def parabola(x):
    return 0.005 * pow(x - 500, 2) + 250

In [89]:
a = Perceptron(3, act='step')
b = Perceptron(3, act='step')
c = Perceptron(3, act='step')

[0.7303820868017199, 0.9273451623463689, -0.0167356293745915]
[0.12043429991342602, 0.16784219372837983, -0.6964690184664923]
[0.9663948928409787, 0.5738521501702445, -0.12094978694234193]


In [90]:
def network(first, second):
    a_out = a.feed_forward([first, second, 1])
    b_out = b.feed_forward([first, second, 1])
    c_out = c.feed_forward([a_out, b_out, 1])
    return c_out

In [97]:
for _ in range(0,1000000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    curve_y = parabola(x_coord)
    x_norm = x_coord / 1000
    y_norm = y_coord / 1000

    c_out = network(x_norm, y_norm)
    
    if curve_y > y_coord:
        answer = 1
    else:
        answer = 0

    back_error = c.backward_pass(c_out - answer)
    a.backward_pass(back_error[0])
    b.backward_pass(back_error[1])
    
print(a.get_weights())
print(b.get_weights())
print(c.get_weights())

[0.3184757921543099, -9.329268559242207, -0.5802042933052338]
[-0.38272658651737856, -884.7666529993556, -0.6218682389443612]
[0.006394892840977955, 0.5738521501702445, 0.009050213057658041]


In [98]:
# determine the accuracy
correct = 0

for _ in range(0,1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    curve_y = parabola(x_coord)
    x_norm = x_coord / 1000
    y_norm = y_coord / 1000
    
    is_above = curve_y > y_coord
    guess_above = network(x_norm, y_norm)
    
    if (is_above == True and guess_above >= 0.5):
        correct += 1
    if (is_above == False and guess_above < 0.5):
        correct += 1

print(correct)

602


In [96]:
print('%f' % c.feed_forward([1, 1, 1]))
print('%f' % c.feed_forward([1, 0, 1]))
print('%f' % c.feed_forward([0, 1, 1]))
print('%f' % c.feed_forward([0, 0, 1]))

1.000000
1.000000
1.000000
1.000000
